In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('bdl20-21data.txt')
winter2 = pd.read_csv ('bdl21-22data.txt')
winter3 = pd.read_csv ('bdl22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
bdl = LocationInfo(
    'Bradley', 'Connecticut', 
    latitude= 41.9389 ,  
    longitude=-72.6860,
    timezone='America/Connecticut'
)

bdl

LocationInfo(name='Bradley', region='Connecticut', timezone='America/Connecticut', latitude=41.9389, longitude=-72.686)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(bdl.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(bdl.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(bdl.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:59:48  2020-12-01 16:19:59  2020-12-01
1   2020-12-02 07:00:50  2020-12-02 16:19:44  2020-12-02
2   2020-12-03 07:01:51  2020-12-03 16:19:32  2020-12-03
3   2020-12-04 07:02:51  2020-12-04 16:19:22  2020-12-04
4   2020-12-05 07:03:50  2020-12-05 16:19:14  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:33:20  2021-02-24 17:35:02  2021-02-24
86  2021-02-25 06:31:48  2021-02-25 17:36:15  2021-02-25
87  2021-02-26 06:30:15  2021-02-26 17:37:27  2021-02-26
88  2021-02-27 06:28:42  2021-02-27 17:38:40  2021-02-27
89  2021-02-28 06:27:07  2021-02-28 17:39:52  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:33  2021-12-01 16:20:03  2021-12-01
1   2021-12-02 07:00:35  2021-12-02 16:19:47  2021-12-02
2   2021-12-03 07:01:37  2021-12-03 16:19:34  2021-12-03
3   2021-12-04 07:02:37  2021-12-04 16:19:24  2021-12-04
4   2021-12-05 07:03:36  2021-12-05 16:19:15  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:33:42  2022-02-24 17:34:44  2022-02-24
86  2022-02-25 06:32:10  2022-02-25 17:35:57  2022-02-25
87  2022-02-26 06:30:38  2022-02-26 17:37:10  2022-02-26
88  2022-02-27 06:29:04  2022-02-27 17:38:22  2022-02-27
89  2022-02-28 06:27:30  2022-02-28 17:39:34  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:18  2022-12-01 16:20:06  2022-12-01
1   2022-12-02 07:00:20  2022-12-02 16:19:50  2022-12-02
2   2022-12-03 07:01:22  2022-12-03 16:19:37  2022-12-03
3   2022-12-04 07:02:22  2022-12-04 16:19:26  2022-12-04
4   2022-12-05 07:03:21  2022-12-05 16:19:17  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:34:04  2023-02-24 17:34:26  2023-02-24
86  2023-02-25 06:32:33  2023-02-25 17:35:39  2023-02-25
87  2023-02-26 06:31:00  2023-02-26 17:36:52  2023-02-26
88  2023-02-27 06:29:27  2023-02-27 17:38:05  2023-02-27
89  2023-02-28 06:27:53  2023-02-28 17:39:17  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDL,2020-12-05 14:51:00,3.5600,RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
1,BDL,2020-12-05 15:51:00,4.0600,RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
2,BDL,2020-12-05 16:51:00,5.3300,-SN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
3,BDL,2020-12-05 17:51:00,3.5600,-SN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
4,BDL,2020-12-05 18:51:00,2.0300,-RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
...,...,...,...,...,...,...,...
176,BDL,2021-02-22 19:51:00,1.5200,-SN BR,2021-02-22,2021-02-22 06:36:21,2021-02-22 17:32:35
177,BDL,2021-02-22 20:51:00,1.7800,-SN BR,2021-02-22,2021-02-22 06:36:21,2021-02-22 17:32:35
178,BDL,2021-02-27 09:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:28:42,2021-02-27 17:38:40
179,BDL,2021-02-27 10:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:28:42,2021-02-27 17:38:40


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDL,2021-12-08 20:51:00,0.0001,-SN,2021-12-08,2021-12-08 07:06:24,2021-12-08 16:19:05
1,BDL,2021-12-08 21:51:00,0.0001,-RASN,2021-12-08,2021-12-08 07:06:24,2021-12-08 16:19:05
2,BDL,2021-12-08 22:51:00,0.5100,-SN BR,2021-12-08,2021-12-08 07:06:24,2021-12-08 16:19:05
3,BDL,2021-12-08 23:51:00,0.2500,-SN BR,2021-12-08,2021-12-08 07:06:24,2021-12-08 16:19:05
4,BDL,2021-12-09 00:51:00,0.7600,-SN BR,2021-12-09,2021-12-09 07:07:17,2021-12-09 16:19:06
...,...,...,...,...,...,...,...
106,BDL,2022-02-25 16:51:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:32:10,2022-02-25 17:35:57
107,BDL,2022-02-25 17:51:00,0.7600,-SN BR,2022-02-25,2022-02-25 06:32:10,2022-02-25 17:35:57
108,BDL,2022-02-25 18:51:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:32:10,2022-02-25 17:35:57
109,BDL,2022-02-25 19:51:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:32:10,2022-02-25 17:35:57


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDL,2022-12-11 15:51:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:46,2022-12-11 16:19:14
1,BDL,2022-12-11 16:51:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:46,2022-12-11 16:19:14
2,BDL,2022-12-11 17:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:46,2022-12-11 16:19:14
3,BDL,2022-12-11 18:51:00,0.2500,-SN,2022-12-11,2022-12-11 07:08:46,2022-12-11 16:19:14
4,BDL,2022-12-11 19:51:00,0.2500,-SN BR,2022-12-11,2022-12-11 07:08:46,2022-12-11 16:19:14
...,...,...,...,...,...,...,...
71,BDL,2023-02-25 19:51:00,0.0001,-SN,2023-02-25,2023-02-25 06:32:33,2023-02-25 17:35:39
72,BDL,2023-02-25 20:51:00,0.0001,-SN,2023-02-25,2023-02-25 06:32:33,2023-02-25 17:35:39
73,BDL,2023-02-25 21:51:00,0.0001,-SN,2023-02-25,2023-02-25 06:32:33,2023-02-25 17:35:39
74,BDL,2023-02-25 22:51:00,0.0001,-SN BR,2023-02-25,2023-02-25 06:32:33,2023-02-25 17:35:39


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDL,2020-12-05 14:51:00,3.5600,RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
1,BDL,2020-12-05 15:51:00,4.0600,RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
7,BDL,2020-12-14 14:51:00,0.2500,-RAPL,2020-12-14,2020-12-14 07:11:28,2020-12-14 16:19:53
8,BDL,2020-12-14 15:51:00,0.0001,-RAPL,2020-12-14,2020-12-14 07:11:28,2020-12-14 16:19:53
17,BDL,2020-12-17 07:51:00,1.5200,-SN BR,2020-12-17,2020-12-17 07:13:30,2020-12-17 16:20:48
...,...,...,...,...,...,...,...
164,BDL,2021-02-19 15:51:00,0.0001,-SN,2021-02-19,2021-02-19 06:40:46,2021-02-19 17:28:52
165,BDL,2021-02-19 16:51:00,0.0001,-SN,2021-02-19,2021-02-19 06:40:46,2021-02-19 17:28:52
178,BDL,2021-02-27 09:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:28:42,2021-02-27 17:38:40
179,BDL,2021-02-27 10:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:28:42,2021-02-27 17:38:40


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
0,BDL,2020-12-05 14:51:00,3.5600,RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14,2020-12-05 08:03:50,2020-12-05 15:19:14
7,BDL,2020-12-14 14:51:00,0.2500,-RAPL,2020-12-14,2020-12-14 07:11:28,2020-12-14 16:19:53,2020-12-14 08:11:28,2020-12-14 15:19:53
18,BDL,2020-12-17 08:51:00,1.2700,-SN BR,2020-12-17,2020-12-17 07:13:30,2020-12-17 16:20:48,2020-12-17 08:13:30,2020-12-17 15:20:48
19,BDL,2020-12-17 09:51:00,1.0200,SN BLSN FZFG,2020-12-17,2020-12-17 07:13:30,2020-12-17 16:20:48,2020-12-17 08:13:30,2020-12-17 15:20:48
20,BDL,2020-12-17 10:51:00,0.5100,-SN BLSN FZFG,2020-12-17,2020-12-17 07:13:30,2020-12-17 16:20:48,2020-12-17 08:13:30,2020-12-17 15:20:48
...,...,...,...,...,...,...,...,...,...
163,BDL,2021-02-19 14:51:00,0.0001,-SN,2021-02-19,2021-02-19 06:40:46,2021-02-19 17:28:52,2021-02-19 07:40:46,2021-02-19 16:28:52
164,BDL,2021-02-19 15:51:00,0.0001,-SN,2021-02-19,2021-02-19 06:40:46,2021-02-19 17:28:52,2021-02-19 07:40:46,2021-02-19 16:28:52
178,BDL,2021-02-27 09:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:28:42,2021-02-27 17:38:40,2021-02-27 07:28:42,2021-02-27 16:38:40
179,BDL,2021-02-27 10:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:28:42,2021-02-27 17:38:40,2021-02-27 07:28:42,2021-02-27 16:38:40


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.22513650793650788
2021-2022: 0.7561884615384616
2022-2023: 0.40655


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,BDL,2020-12-05 16:51:00,5.3300,-SN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
3,BDL,2020-12-05 17:51:00,3.5600,-SN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
4,BDL,2020-12-05 18:51:00,2.0300,-RASN BR,2020-12-05,2020-12-05 07:03:50,2020-12-05 16:19:14
5,BDL,2020-12-09 23:51:00,0.0001,-SN,2020-12-09,2020-12-09 07:07:29,2020-12-09 16:19:07
6,BDL,2020-12-10 00:51:00,0.0001,-SN,2020-12-10,2020-12-10 07:08:20,2020-12-10 16:19:12
...,...,...,...,...,...,...,...
173,BDL,2021-02-20 00:51:00,0.2500,-SN BR,2021-02-20,2021-02-20 06:39:19,2021-02-20 17:30:06
174,BDL,2021-02-22 17:51:00,0.0001,-SN,2021-02-22,2021-02-22 06:36:21,2021-02-22 17:32:35
175,BDL,2021-02-22 18:51:00,0.7600,-SN BR,2021-02-22,2021-02-22 06:36:21,2021-02-22 17:32:35
176,BDL,2021-02-22 19:51:00,1.5200,-SN BR,2021-02-22,2021-02-22 06:36:21,2021-02-22 17:32:35


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_3876\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 1.0527291666666667
2021-2022: 0.28171111111111113
2022-2023: 0.40755


In [65]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'CT'

data.to_csv('SO.bdl.csv', index=False)